In [ ]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import pandas as pd
import PIL
import sklearn

def load_data_train():
    return pd.read_csv("../input/gtsrb-german-traffic-sign/Train.csv")

train_df = load_data_train()
train_df["ClassId"] = train_df["ClassId"].astype("str")
seed = 12
from sklearn.utils import shuffle
train_df = shuffle(train_df)  # Dataframe must be shuffled if you use flow_from_dataframe!


In [ ]:
imgdatagen_train = preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=False,
    vertical_flip=False,
    validation_split=0.2,
)
imgdatagen_val = preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)

train_ds = imgdatagen_train.flow_from_dataframe(train_df, directory="../input/gtsrb-german-traffic-sign/", x_col="Path", y_col="ClassId",
                                          target_size=(32, 32), subset="training", seed=seed, batch_size=512)
val_ds = imgdatagen_val.flow_from_dataframe(train_df, directory="../input/gtsrb-german-traffic-sign/", x_col="Path", y_col="ClassId",
                                          target_size=(32, 32), subset="validation", seed=seed, batch_size=512)

#%%


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
cur = next(val_ds)
cur_t = next(train_ds)

lookingfor = np.zeros(43)
lookingfor[0] = 1

for i, y in enumerate(cur[1]):
    if np.all(y == lookingfor):
        print(y)
        plt.imshow(cur[0][i])
        plt.show()
for i, y in enumerate(cur_t[1]):
    if np.all(y == lookingfor):
        print(y)
        plt.imshow(cur_t[0][i])
        plt.show()

In [ ]:
model = tf.keras.Sequential([
    layers.InputLayer((32, 32, 3)),
    layers.Conv2D(32, 3, activation="relu",
                  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    layers.BatchNormalization(),
    layers.Conv2D(64, 3, activation="relu",
                  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(256, 3, activation="relu",
                  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(1024, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1024, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(nbclass, activation="softmax")
]
)

#%%

print(model.summary())

In [ ]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
hist = model.fit(train_ds, validation_data=val_ds, epochs=50)

In [ ]:
testdf = pd.read_csv("../input/gtsrb-german-traffic-sign/Test.csv")
testdf["ClassId"] = testdf["ClassId"].astype("str")
test_ds = imgdatagen_val.flow_from_dataframe(testdf, directory="../input/gtsrb-german-traffic-sign/", x_col="Path", y_col="ClassId",
                                          target_size=(32, 32), seed=seed, batch_size=512)


In [ ]:
model.evaluate(test_ds)